## Постановка задачи

### PFound
Исходные данные - Yandex Cup 2022 Analytics
- Ссылка - https://yandex.ru/cup/analytics/analysis/ , пример A. Рассчитать pFound
- Данные - https://yadi.sk/d/guqki4UI4hFlXQ
- Формула
$$pFound@K = \sum_{i=1}^{k} pLook[i]\ pRel[i]$$

$$pLook[1] = 1$$

$$pLook[i] = pLook[i-1]\ (1 - pRel[i-1])\ (1 - pBreak)$$

$$pBreak = 0.15$$

**Задача** - написать функцию, которая принимает на вход dataframe (после join), а на выходе дает средний pFound по всем query.
- Запрещается использовать циклы for для расчет метрики (как полностью, так и ее частей).
- Усложнение, если задача показалась легкой - попробуйте обойтись без groupby (не уверен, что это возможно, но вдруг вы справитесь)

## Импорты

In [ ]:
import pandas as pd

## Подготовка данных 

In [ ]:
!mkdir ../data/hw2-2/

In [ ]:
import requests
from urllib.parse import urlencode

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://yadi.sk/d/guqki4UI4hFlXQ'

# Получаем загрузочную ссылку
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

# Загружаем файл и сохраняем его
download_response = requests.get(download_url)
with open('../data/hw2-2/data.zip', 'wb') as f:   # Здесь укажите нужный путь к файлу
    f.write(download_response.content)

In [ ]:
!unzip yandex_cup_analytics_A.zip -d "../data/hw2-2"
!unzip "../data/hw2-2/open_task.zip" -d "../data/hw2-2"

Archive:  yandex_cup_analytics_A.zip
  inflating: ../data/hw2/hidden_task.zip  
  inflating: ../data/hw2/open_task.zip  
Archive:  ../data/hw2/open_task.zip
   creating: ../data/hw2/open_task/
  inflating: ../data/hw2/open_task/qid_query.tsv  
  inflating: ../data/hw2/open_task/hostid_url.tsv  
  inflating: ../data/hw2/open_task/qid_url_rating.tsv  


In [ ]:
qid_query = pd.read_csv("../data/hw2-2/open_task/qid_query.tsv", sep="\t", names=["qid", "query"])
qid_url_rating = pd.read_csv("../data/hw2-2/open_task/qid_url_rating.tsv", sep="\t", names=["qid", "url", "rating"])
hostid_url = pd.read_csv("../data/hw2-2/open_task/hostid_url.tsv", sep="\t", names=["hostid", "url"])

qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")

In [ ]:
qid_url_rating_hostid

,qid,url,rating,hostid
0,10387,http://batman-arkhamcity.ru/,0.00,64
1,10387,http://bigtorrents.org/publ/batman_arkham_city...,0.14,71
2,10387,http://consolelife.ru/xbox-360/6577-o-rossiysk...,0.14,101
3,10387,http://dic.academic.ru/book.nsf/3662736/Batman...,0.00,115
4,10387,http://forum.csmania.ru/viewtopic.php?t=25986,0.14,155
...,...,...,...,...
849,380923,http://www.spishy.ru/referats/14/8040,0.00,1063
850,380923,http://www.sprinter.ru/books/1821345.html,0.00,1065
851,380923,http://www.twirpx.com/files/mathematics/algebr...,0.00,1105
852,380923,http://wzyocgeawwo.eklablog.com/2-a60036325,0.00,1168


## Оригинальное решение 
источник: [Задачи пробного тура чемпионата по програм­мированию  трека «Аналитика»](https://yandex.ru/cup/analytics/analysis/)

### Функция

In [ ]:
def pfound_cicle(data, k=10, pbreak = 0.15):
    def plook(ind, rels):
        if ind == 0:
            return 1
        return plook(ind-1, rels)*(1-rels[ind-1])*(1-pbreak)

    def pfound(group):
        max_by_host = group.groupby("hostid")["rating"].max() # максимальный рейтинг хоста
        topk = max_by_host.sort_values(ascending=False)[:k] # берем топ10 урлов с наивысшим рейтингом
        pfound = 0
        for ind, val in enumerate(topk):
            pfound += val*plook(ind, topk.values)
        return pfound

    qid_pfound = data.groupby('qid').apply(pfound)
    return qid_pfound 

### Тестирование времени работы

In [ ]:
%timeit pfound_cicle(qid_url_rating_hostid)

17.7 ms ± 291 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Подсчет результатов на данной выборке

In [ ]:
base_result = pfound_cicle(qid_url_rating_hostid)
base_result

qid
10387     0.497771
20860     0.655448
21070     0.497771
35618     0.437794
107538    0.354808
150126    0.366109
168170    0.481255
176370    0.393661
192007    0.191170
213932    0.347005
221830    0.497771
242953    0.497771
253476    0.497771
295761    0.900836
346214    0.263596
347852    0.618534
360100    0.470204
366042    0.309314
375608    0.497771
380923    0.429989
dtype: float64

## Решение с использованием Pandas

### Функция

In [ ]:
def pfound_pandas(data, k = 10, pbreak = 0.15):
    # Получение максимального элемента для каждого значения hostid в подгруппе qid
    max_by_qid_host = data.groupby(["qid", "hostid"])["rating"].max().reset_index()
    # Cортировка значений 
    max_by_qid_host = max_by_qid_host.sort_values(["qid", "rating"], ascending=False)
    # Получение K лучших рейтингов в рамках одного qid-а
    head_for_qid = max_by_qid_host.groupby("qid").head(k).reset_index(drop=True)
    # Подсчет вспомогательного значения 
    head_for_qid["product"] = (1 - head_for_qid['rating']).shift(1) * (1 - pbreak) 
    # Инициализация базы рекурсии (pLook[1] = 1)
    head_for_qid.loc[head_for_qid.index % k == 0, "product"] = 1
    # Рекурсивный подсчет значения pLook в рамках каждой группы
    head_for_qid["plook"] = head_for_qid.groupby("qid")["product"].cumprod()
    # Подсчет слагаемых в формуле pFound 
    head_for_qid["pfound_term"] = head_for_qid["plook"] * head_for_qid["rating"]
    # Подсчет pFound
    pfound = head_for_qid.groupby("qid")["pfound_term"].sum()
    return pfound

### Тестирование времени работы

In [ ]:
%timeit pfound_pandas(qid_url_rating_hostid)

8.78 ms ± 134 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Подсчет результатов на данной выборке

In [ ]:
pandas_result = pfound_pandas(qid_url_rating_hostid)
pandas_result

qid
10387     0.497771
20860     0.655448
21070     0.497771
35618     0.437794
107538    0.354808
150126    0.366109
168170    0.481255
176370    0.393661
192007    0.191170
213932    0.347005
221830    0.497771
242953    0.497771
253476    0.497771
295761    0.900836
346214    0.263596
347852    0.618534
360100    0.470204
366042    0.309314
375608    0.497771
380923    0.429989
Name: pfound_term, dtype: float64

## Сравнение результатов работы двух методов

In [ ]:
assert (abs(pandas_result - base_result) < 1e-9).all(), "WA"